In [1]:
%run ../ipynb_util_tars.py

In [2]:
%env WANDB_PROJECT=thesis

env: WANDB_PROJECT=thesis


In [3]:
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# LLaMA3 8B

## Load ZO_UP data

In [4]:
%run ../ipynb_load_data.py

{'SDG': ClassLabel(names=['1', '10', '11', '12', '13', '14', '15', '16', '17', '2', '3', '4', '5', '6', '7', '8', '9'], id=None), 'ABSTRACT': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None), 'sdg_desc_short': Value(dtype='string', id=None), 'sdg_desc_long': Value(dtype='string', id=None)}
Example instance:	 {'SDG': 16, 'ABSTRACT': 'The first attempts to modernize simply replaced the single huge engine with a huge electric motor, changing little. The drive-shafts were replaced by wires, the huge steam engine by dozens of small motors. Factories spread out, there was natural light, and room to use ceiling-slung cranes. Workers had responsibility for their own machines, they needed better training and better pay. The electric motor was a wonderful invention, once we changed all the everyday details that surrounded it.', 'id': None, 'sdg_desc_short': None, 'sdg_desc_long': None}
Encoded (label2id) label:	 16
Decoded (id2label) label:	 9
9 16 16


In [5]:
import pprint
pprint.pprint(id2label)

{0: '1',
 1: '10',
 2: '11',
 3: '12',
 4: '13',
 5: '14',
 6: '15',
 7: '16',
 8: '17',
 9: '2',
 10: '3',
 11: '4',
 12: '5',
 13: '6',
 14: '7',
 15: '8',
 16: '9'}


## Model

In [6]:
from transformers.data import DataCollatorWithPadding
from transformers.models.llama import LlamaForSequenceClassification
from peft import LoraConfig, TaskType, get_peft_model

MODEL_NAME = "meta-llama/Meta-Llama-3-8B"
#MODEL_NAME = "TinyLlama/TinyLlama_v1.1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    num_labels=len(labels),
    token=HF_TOKEN
).bfloat16()
model.config.pad_token_id = tokenizer.pad_token_id
# model.resize_token_embeddings(len(tokenizer))

# Add PEFT LoRA here
# from peft import LoraConfig, TaskType, get_peft_model
# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_CLS,
#     inference_mode=False,
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.1
# )
# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

encoded_dataset = dataset.map(
    preprocess_data(tokenizer), batched=True, remove_columns=dataset["train"].column_names
)
encoded_dataset.set_format("torch")

data_collator = DataCollatorWithPadding(tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <enum 'DatasetType'>.
  StockPickler.save(self, obj, save_persistent_id)
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <enum 'DatasetType'>: __main__.DatasetType has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


In [7]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer, EvalPrediction

BATCH_SIZE = 8
METRIC_NAME = "accuracy"

args = TrainingArguments(
    f"{CHECKPOINT_PATH}/{MODEL_NAME}-fullft-{DATASET_TYPE.value}",
    evaluation_strategy="steps",
    logging_strategy="steps",
    save_strategy="steps",
    eval_steps=50,
    #save_strategy="steps",
    save_steps=50,
    logging_steps=50,
    learning_rate=5e-3,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=20,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model=METRIC_NAME,
    seed=SEED,
    report_to="wandb",
)

def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    accuracy = accuracy_score(labels, pred.predictions.argmax(-1))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred.predictions.argmax(-1), average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dvdblk (ngmi). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,59.206200,34.616238,0.062731,0.003935,0.062731,0.007406
100,25.633700,18.807657,0.062731,0.003935,0.062731,0.007406
150,17.361200,14.808810,0.062731,0.003935,0.062731,0.007406
200,14.836900,12.487777,0.062731,0.003935,0.062731,0.007406
250,9.804400,6.596748,0.062731,0.003935,0.062731,0.007406
300,7.736900,6.966329,0.062731,0.003935,0.062731,0.007406
350,5.754400,5.175508,0.062731,0.003935,0.062731,0.007406
400,5.177800,4.419857,0.062731,0.003935,0.062731,0.007406
450,4.570000,4.416052,0.062731,0.003964,0.062731,0.007457
500,4.448800,3.560251,0.062731,0.008716,0.062731,0.011482


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

TrainOutput(global_step=1580, training_loss=6.93407832278481, metrics={'train_runtime': 10257.5509, 'train_samples_per_second': 1.228, 'train_steps_per_second': 0.154, 'total_flos': 2.701630076092416e+17, 'train_loss': 6.93407832278481, 'epoch': 20.0})

In [9]:
trainer.evaluate()

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 3.2491352558135986,
 'eval_accuracy': 0.1070110701107011,
 'eval_precision': 0.05136157112962596,
 'eval_recall': 0.1070110701107011,
 'eval_f1': 0.0578771249250954,
 'eval_runtime': 32.3302,
 'eval_samples_per_second': 8.382,
 'eval_steps_per_second': 1.052,
 'epoch': 20.0}

In [10]:
from sklearn.metrics import classification_report

model.eval()

# manual evaluation to show classifcation_report
true_labels = []
logits = []

for batch in encoded_dataset["test"]:
    batch = {k: v.to(trainer.args.device).unsqueeze(0) for k, v in batch.items()}
    label = batch.pop("label")

    # Forward pass
    with torch.no_grad():
        out = model(**batch)

    true_labels.append(label.item())
    logits.extend(out.logits.tolist())

probabilites = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
pred_labels = torch.argmax(probabilites, dim=-1).tolist()

report = classification_report(true_labels, pred_labels, target_names=[f"SDG {id2label[i]}" for i in range(len(labels))])

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

In [11]:
print(report)

              precision    recall  f1-score   support

       SDG 1       0.00      0.00      0.00        17
      SDG 10       0.00      0.00      0.00        17
      SDG 11       0.08      0.41      0.13        17
      SDG 12       0.00      0.00      0.00        17
      SDG 13       0.00      0.00      0.00        17
      SDG 14       0.00      0.00      0.00        17
      SDG 15       0.22      0.65      0.32        17
      SDG 16       0.00      0.00      0.00        17
      SDG 17       0.00      0.00      0.00         1
       SDG 2       0.04      0.12      0.06        16
       SDG 3       0.00      0.00      0.00        17
       SDG 4       0.00      0.00      0.00        17
       SDG 5       0.00      0.00      0.00        17
       SDG 6       0.09      0.29      0.14        17
       SDG 7       0.38      0.18      0.24        17
       SDG 8       0.00      0.00      0.00        16
       SDG 9       0.00      0.00      0.00        17

    accuracy              